In [81]:
import pandas as pd
import re
from src.Remove_accents import remove_accents

In [82]:
df=pd.read_csv('Databases/Cleaned_df.csv')

In [83]:
df["inalambrico"]=df["inalambrico"].apply(lambda x: remove_accents(str(x)).lower())
df.loc[df["inalambrico"]=="nan","inalambrico"]="no"
df.loc[df["inalambrico"]=="inalambrico","inalambrico"]="si"
df=df[df["inalambrico"]=="si"].reset_index(drop=True).drop(columns="inalambrico")

In [84]:
def parse_weight(row):
    """
    Convierte una cadena de peso en kilogramos.
    """
    # Limpieza inicial y conversión a minúsculas
    weight = str(row).strip().lower()
    
    # Expresiones regulares para identificar peso y unidad
    weight_match = re.search(r"([\d.,]+)\s*(kg|kilogramos|kilos|g|gramos|libras|libra)?", weight)
    if not weight_match:
        return None  # Si no se puede parsear, retorna None
    
    # Obtener valor numérico y unidad
    value = float(weight_match.group(1).replace(',', '.'))
    unit = weight_match.group(2)
    
    # Conversión según unidad
    if unit in ("kg", "kilogramos", "kilos", None):
        return value
    elif unit in ("g", "gramos"):
        return value / 1000  # gramos a kilogramos
    elif unit in ("libras", "libra"):
        return value * 0.453592  # libras a kilogramos
    else:
        return None  # Si no se reconoce la unidad, retorna None

df["peso"]=df["peso"].apply(parse_weight)

In [85]:
def parse_warranty(row):
    """
    Convierte una cadena de garantía en años (float).
    Retorna None si no hay tiempo explícito.
    """
    # Limpieza inicial y conversión a minúsculas
    warranty = str(row).strip().lower()
    
    # Expresión regular ajustada para capturar solo tiempos explícitos
    warranty_match = re.search(r"(\d+)\s*(años|año|year|years|meses|mes|months|month)", warranty)
    if not warranty_match:
        return None  # Si no se puede parsear, retorna None
    
    # Obtener valor numérico y unidad
    value = int(warranty_match.group(1))
    unit = warranty_match.group(2)
    
    # Conversión según unidad
    if unit in ("años", "año", "year", "years"):
        return value  # Ya está en años
    elif unit in ("meses", "mes", "months", "month"):
        return value / 12  # Meses a años
    else:
        return None  # Redundante, pero asegura robustez

df["garantia"]=df["garantia"].apply(parse_warranty)

In [86]:
def parse_noise_level(row):
    """
    Extrae el nivel de ruido como número (float).
    Retorna None si no se puede extraer un valor válido.
    """
    # Expresión regular para capturar números, incluyendo decimales
    noise_match = re.search(r"(\d+(?:[.,]?\d*)?)\s*(d[bB]|decibelios|dBA)?", str(row).strip())
    if not noise_match:
        return None  # Si no hay un número válido, retorna None
    
    # Reemplazar comas por puntos para manejar decimales
    noise_value = noise_match.group(1).replace(',', '.')
    
    return float(noise_value)

df["nivel de ruido"]=df["nivel de ruido"].apply(parse_noise_level)

In [87]:
def parse_cable_length(row):
    """
    Extrae la longitud del cable en metros (float).
    Retorna None si es inalámbrico o no especificado.
    """
    cable = str(row).strip().lower()

    # Identificar palabras clave para "inalámbrico"
    if any(keyword in cable for keyword in ["inalámbrico", "sin cable", "no especificado", "no disponible"]):
        return None
    
    # Expresión regular para capturar longitudes y unidades
    match = re.search(r"(\d+(?:[.,]?\d*)?)\s*(cm|centímetros|m|metros)?", cable)
    if not match:
        return None  # Si no se encuentra un valor válido
    
    # Obtener el valor numérico y la unidad
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2)
    
    # Convertir a metros si está en centímetros
    if unit in ["cm", "centímetros"]:
        return value / 100
    elif unit in ["m", "metros"]:
        return value
    else:
        return value  # Sin unidad explícita, asumir como metros

df["longitud cable"]=df["longitud cable"].apply(parse_cable_length)
df=df[df["longitud cable"].isna()].reset_index(drop=True)

In [88]:
def parse_material(row):
    """
    Estándariza el material en categorías como 'Plástico', 'Metal', 'ABS', etc.
    Retorna 'No especificado' si no es posible determinar el material.
    """
    material = remove_accents(str(row)).strip().lower()

    # Identificar palabras clave y mapear a categorías
    if "plástico" in material or "plastico" in material or "abs" in material or "pp" in material:
        return "Plástico"
    if "metal" in material or "acero" in material or "aleación" in material or "aluminio" in material:
        return "Metal"
    if "fibra de carbono" in material:
        return "Fibra de carbono"
    if "combinación" in material or "/" in material or "," in material:
        return "Combinación"
    if "no especificado" in material or "no disponible" in material or material in ["default", "upgrade"] or pd.notna(material):
        return "No especificado"

    # Si no coincide con categorías conocidas
    return "Otro"

df["material"]=df["material"].apply(parse_material)


In [89]:
def parse_bateria(row):
    """
    Analiza la información de las baterías para estandarizar:
    - Capacidad en mAh
    - Voltaje en Voltios
    - Características adicionales
    """
    data = str(row).lower()
    result = {"capacidad_mAh": None, "voltaje_V": None, "extra_info": None}
    
    # Extraer capacidad en mAh
    capacidad_match = re.search(r'(\d+(\.\d+)?\s*(mAh|mah|ah))', data)
    if capacidad_match:
        result["capacidad_mAh"] = float(re.search(r'\d+', capacidad_match.group()).group())

    # Extraer voltaje en V
    voltaje_match = re.search(r'(\d+(\.\d+)?\s*(v|voltios|volt))', data)
    if voltaje_match:
        result["voltaje_V"] = float(re.search(r'\d+(\.\d+)?', voltaje_match.group()).group())

    # Características adicionales
    if "recargable" in data:
        result["extra_info"] = "recargable"
    elif "extraible" in data or "desmontable" in data:
        result["extra_info"] = "extraíble"
    elif "iones de litio" in data or "litio" in data:
        result["extra_info"] = "iones de litio"
    else:
        result["extra_info"] = "otra"

    return result

# No dan mucha info
# df_parsed = df['bateria'].apply(parse_bateria)
# df_parsed = pd.json_normalize(df_parsed) 

In [90]:
def categorize_value(value):
    if pd.isna(value):
        return "Unknown"
    value = remove_accents(str(value).lower())
    if "bateria" in value or "battery" in value or "电池" in value:
        return "Battery-powered"
    if "usb" in value:
        return "USB-powered"
    if "electrico" in value or "electric" in value:
        return "Electric-powered"
    if "voltios" in value or re.search(r'\d+v', value):
        return "Voltage-specific"
    if "neumatica" in value:
        return "Pneumatic-powered"
    if "pilas" in value:
        return "Battery-powered"
    if "cable" in value:
        return "Cable-powered"
    if "no especificado" in value:
        return "Not specified"
    return "Other"
    
df["alimentacion"]=df["alimentacion"].apply(categorize_value).value_counts()

In [91]:
import re
import pandas as pd
import numpy as np

def extract_bag_volume(row, columns, priority_order):
    """
    Extracts the volume of the bag in liters based on the priority order of columns.
    Assumes the priority_order specifies the precedence of columns for extracting the volume.

    Parameters:
    - row: A single row of a DataFrame (as a Series).
    - columns: A list of column names to check for volume information.
    - priority_order: A list of column names in order of priority.

    Returns:
    - A float representing the volume in liters, or NaN if no volume is found.
    """
    for col in priority_order:
        if col in row and pd.notna(row[col]):
            text = row[col]
            # Normalize text: lowercase, replace common synonyms, etc.
            text = str(text).lower()
            text = text.replace(",", ".")
            
            # Match volume patterns (e.g., "1.5L", "500 ml", "0.9 litros", etc.)
            match = re.search(r'(\d+(?:\.\d+)?)\s*(litros?|l|ml|mililitros?)', text)
            if match:
                volume, unit = match.groups()
                volume = float(volume)
                # Convert to liters if unit is milliliters
                if 'ml' in unit or 'mililitros' in unit:
                    volume /= 1000
                return volume
    return np.nan

priority_order = ['capacidad del deposito de polvo','bolsa', 'almacenamiento']

df['volume_litres'] = df.apply(extract_bag_volume, axis=1, columns=df.columns, priority_order=priority_order)

In [92]:
import re
import pandas as pd

def classify_brush_and_head(row):
    """
    Classifies information from the 'cepillo' and 'cabezal' columns into structured categories.

    Parameters:
    - row (pd.Series): A single row containing 'cepillo' and 'cabezal'.

    Returns:
    - A dictionary with combined classification:
        - 'N_total': Total count of brushes and heads (Single/Multiple).
        - 'type_combined': Combined types of brushes and heads (General Purpose, Anti-Tangle, etc.).
        - 'features_combined': Combined features (Rotating, Flexible, LED, etc.).
    """
    def classify_component(text, key_prefix):
        """
        Parses a single text component (cepillo or cabezal).
        """
        if pd.isna(text):
            return {f"{key_prefix}_N": None, f"{key_prefix}_type": None, f"{key_prefix}_features": []}

        text = str(text).lower()  # Normalize text
        result = {f"{key_prefix}_N": None, f"{key_prefix}_type": None, f"{key_prefix}_features": []}

        # Determine quantity
        quantity_match = re.search(r'\b(\d+)\s*(cepillos?|cabezales?|tipos?)\b', text)
        if quantity_match:
            result[f"{key_prefix}_N"] = "Multiple" if int(quantity_match.group(1)) > 1 else "Single"
        else:
            result[f"{key_prefix}_N"] = "Single"

        # Classify types
        type_keywords = [
            "antienredos", "giratorio", "de rodillo", "turbo", "multifunción", "para polvo"
        ]
        type_match = [word for word in type_keywords if word in text]
        result[f"{key_prefix}_type"] = ", ".join(type_match) if type_match else "General Purpose"

        # Extract features
        features_keywords = {
            "rotating": ["giratorio", "rotatorio"],
            "flexible": ["flexible", "ajustable", "pivotante"],
            "led": ["led"],
            "motorized": ["motorizado", "motor"],
            "removable": ["extraíble", "desmontable"],
        }
        for feature, keywords in features_keywords.items():
            if any(word in text for word in keywords):
                result[f"{key_prefix}_features"].append(feature.capitalize())

        return result

    # Classify 'cepillo' and 'cabezal'
    cepillo_data = classify_component(row['cepillo'], "cepillo")
    cabezal_data = classify_component(row['cabezal'], "cabezal")

    # Combine results
    combined = {
        "N_cepillos": "Multiple" if "Multiple" in [cepillo_data["cepillo_N"], cabezal_data["cabezal_N"]] else "Single",
        "type_cepillo": ", ".join(
            filter(None, {cepillo_data["cepillo_type"], cabezal_data["cabezal_type"]})
        ),
        "features_cepillo": list(set(cepillo_data["cepillo_features"] + cabezal_data["cabezal_features"])),
    }

    return combined

# Apply the updated function

df = pd.concat([df, pd.json_normalize(df.apply(classify_brush_and_head, axis=1))], axis=1)

In [93]:
import re
import pandas as pd
import numpy as np

def parse_charging_time(text):
    """
    Parses the `tiempo de carga` column to extract charging times in hours.

    Parameters:
    - text (str): A single cell value from the `tiempo de carga` column.

    Returns:
    - A float representing the average charging time in hours, or NaN for invalid entries.
    """
    if pd.isna(text):
        return np.nan

    text = str(text).lower().strip()
    text = text.replace(',', '.')  # Replace commas with dots for float parsing

    # Match exact times or ranges (e.g., "4 horas", "2.5-3 horas")
    range_match = re.match(r'(\d+(\.\d+)?)\s*(?:-|~|a|to)\s*(\d+(\.\d+)?)\s*horas?', text)
    if range_match:
        # Calculate the average of the range
        start, end = float(range_match.group(1)), float(range_match.group(3))
        return (start + end) / 2

    # Match single numeric values with "horas" or "minutos"
    hour_match = re.match(r'(\d+(\.\d+)?)\s*horas?', text)
    if hour_match:
        return float(hour_match.group(1))

    minute_match = re.match(r'(\d+)\s*minutos?', text)
    if minute_match:
        return float(minute_match.group(1)) / 60  # Convert minutes to hours

    # Handle special cases
    if "menos de" in text:
        less_than_match = re.match(r'menos de (\d+(\.\d+)?)\s*horas?', text)
        if less_than_match:
            return float(less_than_match.group(1)) * 0.9  # Approximate to 90% of the value

    # If no valid match is found, return NaN
    return np.nan

df['tiempo de carga']=df['tiempo de carga'].apply(parse_charging_time)

In [94]:
import re
import pandas as pd

def classify_applications(text):
    """
    Classifies the `aplicaciones` column into structured categories.

    Parameters:
    - text (str): A single cell value from the `aplicaciones` column.

    Returns:
    - A dictionary with classification:
        - 'location': Categories like 'Home', 'Automobile', 'Office', or 'Other Locations'.
        - 'surface_types': Categories like 'Hard Floors', 'Carpets', 'Upholstery', etc.
        - 'special_features': Categories like 'Pet Hair', 'Detailed Cleaning', etc.
    """
    if pd.isna(text):
        return {
            "location": None,
            "surface_types": None,
            "special_features": None
        }
    
    text = str(text).lower()  # Normalize text
    result = {
        "location": [],
        "surface_types": [],
        "special_features": []
    }
    
    # Classify locations
    if any(word in text for word in ["casa", "hogar", "limpieza doméstica", "vivienda"]):
        result["location"].append("Home")
    if any(word in text for word in ["coche", "automóvil", "vehículo", "carro"]):
        result["location"].append("Automobile")
    if "oficina" in text:
        result["location"].append("Office")
    if any(word in text for word in ["jardines", "viajes", "interior/exterior"]):
        result["location"].append("Other Locations")
    
    # Classify surface types
    if any(word in text for word in ["suelo duro", "baldosas", "mármol", "madera dura"]):
        result["surface_types"].append("Hard Floors")
    if any(word in text for word in ["alfombra", "moqueta"]):
        result["surface_types"].append("Carpets")
    if any(word in text for word in ["tapicería", "sofás", "muebles"]):
        result["surface_types"].append("Upholstery")
    if any(word in text for word in ["colchones", "camas"]):
        result["surface_types"].append("Beds")
    if any(word in text for word in ["ventanas", "teclados", "bordes", "esquinas"]):
        result["surface_types"].append("Other Surfaces")
    
    # Classify special features
    if any(word in text for word in ["pelo de mascotas", "pelos de animales"]):
        result["special_features"].append("Pet Hair")
    if any(word in text for word in ["detallada", "esquinas", "bordes", "rincones difíciles"]):
        result["special_features"].append("Detailed Cleaning")
    if any(word in text for word in ["seco y húmedo", "húmedo"]):
        result["special_features"].append("Wet/Dry")
    
    # Convert lists to strings or None if empty
    for key in result:
        result[key] = ", ".join(result[key]) if result[key] else None
        # result[key].replace("None ,","").replace(", None","")
    return result

# Expand the classification dictionary into separate columns
df=pd.concat([df,pd.json_normalize(df['aplicaciones'].apply(classify_applications))],axis=1)
df=df[df["location"]!="Automobile"].reset_index(drop=True).drop(columns=["location","special_features"])

In [95]:
def parse_funcion_seca_suelo(text):
    """
    Parses the `funcion seca suelo` column to determine if the vacuum has the dry floor function.
    Leverages `remove_accents` for text normalization.
    
    Parameters:
    - text (str): A single cell value from the `funcion seca suelo` column.
    
    Returns:
    - str: 'Yes' if the function is present, 'No' otherwise.
    """
    if pd.isna(text):
        return "No"  # Default to 'No' if the entry is missing
    
    # Use remove_accents to normalize text
    text = remove_accents(str(text).lower().strip())
    
    # Check for explicit 'no'
    if re.search(r'\bno\b', text):
        return "No"
    
    # Check for 'si', 'sí', or mentions of 'succion'
    if re.search(r'\bsi\b', text) or 'succion' in text:
        return "Yes"
    
    # Default classification
    return "No"

df['funcion seca suelo']=df['funcion seca suelo'].apply(parse_funcion_seca_suelo)

In [96]:
def extract_pressure(text):
    if pd.isna(text):
        return None
    
    text = str(text).lower()
    text = text.replace(',', '.').replace(".000","000")  # Normalize decimal points
    # Match single or range values in Pa, kPa, or mmHg
    match = re.search(r'(\d+(?:\.\d+)?)(?:[-–](\d+(?:\.\d+)?))?\s*(pa|kpa|mmhg)', text)
    if match:
        value = float(match.group(1))
        if match.group(2):  # Handle ranges by calculating the midpoint
            value = (value + float(match.group(2))) / 2
        unit = match.group(3)
        if unit == "pa":
            return value / 1000  # Convert Pa to kPa
        elif unit == "mmhg":
            return value * 0.133322  # Convert mmHg to kPa
        return value  # Already in kPa
    
    return None
def parse_suction_pressure(row):
    """
    Parses suction pressure values from 'maxima presion de succion' or 'funcion seca suelo'.
    Converts all values to kPa, including mmHg.

    Parameters:
    - row (pd.Series): A single row from the DataFrame containing both columns.

    Returns:
    - float: Suction pressure in kPa, or NaN if no valid value is found.
    """
    # Attempt to parse 'maxima presion de succion'
    suction_pressure = extract_pressure(row['maxima presion de succion'])

    # Fallback to 'funcion seca suelo' if necessary
    if suction_pressure is None:
        suction_pressure = extract_pressure(row['funcion seca suelo'])

    return suction_pressure

# Apply the parsing function
df['suction_pressure_kpa'] = df.apply(parse_suction_pressure, axis=1)
df.loc[df["suction_pressure_kpa"].isna(),"suction_pressure_kpa"]=df.loc[df["suction_pressure_kpa"].isna(),'All_info'].apply(extract_pressure)
df.loc[df["suction_pressure_kpa"].isna(),"suction_pressure_kpa"]=df.loc[df["suction_pressure_kpa"].isna(),'caracteristicas adicionales'].apply(extract_pressure)

C:\Users\Rafa\AppData\Local\Temp\ipykernel_27940\683565098.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df["suction_pressure_kpa"].isna(),"suction_pressure_kpa"]=df.loc[df["suction_pressure_kpa"].isna(),'caracteristicas adicionales'].apply(extract_pressure)


In [97]:
import pandas as pd
import re

# Supongamos que tu DataFrame se llama df
def extract_max_minutes(s):
    """
    Extrae todos los valores de tiempo en minutos de una cadena de texto y retorna el valor máximo.
    
    Parámetros:
    s (str): Cadena de texto que contiene información sobre la autonomía.
    
    Retorna:
    int: El valor máximo de autonomía en minutos. Retorna 0 si no se encuentra ningún valor válido.
    """
    if not isinstance(s, str):
        return 0

    # Convertir la cadena a minúsculas para facilitar las búsquedas
    s = s.lower()

    # Lista para almacenar todos los valores de minutos encontrados
    matches = []

    # 1. Buscar y procesar patrones combinados "X hora y Y min"
    pattern_combined = re.compile(r'(\d+)\s*hora[s]?\s*y\s*(\d+)\s*min(?:uto[s]?)?')
    for match in pattern_combined.finditer(s):
        horas = int(match.group(1))
        minutos = int(match.group(2))
        total = horas * 60 + minutos
        matches.append(total)

    # Eliminar los patrones combinados de la cadena para evitar doble conteo
    s = pattern_combined.sub('', s)

    # 2. Buscar y procesar rangos "X-Y min" o "X-Y horas"
    pattern_range = re.compile(r'(\d+)\s*-\s*(\d+)\s*(hora[s]?|min(?:uto[s]?)?)')
    for match in pattern_range.finditer(s):
        num1 = int(match.group(1))
        num2 = int(match.group(2))
        unit = match.group(3)

        if 'hora' in unit:
            matches.append(num1 * 60)
            matches.append(num2 * 60)
        else:
            matches.append(num1)
            matches.append(num2)

    # Eliminar los rangos de la cadena para evitar doble conteo
    s = pattern_range.sub('', s)

    # 3. Buscar y procesar valores individuales "X hora(s)" o "X min"
    pattern_single = re.compile(r'(\d+)\s*(hora[s]?|min(?:uto[s]?)?)')
    for match in pattern_single.finditer(s):
        num = int(match.group(1))
        unit = match.group(2)

        if 'hora' in unit:
            matches.append(num * 60)
        else:
            matches.append(num)

    # Retornar el valor máximo encontrado, o 0 si no se encontró ninguno
    if matches:
        return max(matches)
    else:
        return 0

# Aplicar la función a la columna 'autonomía'
df['autonomia'] = df['autonomia'].apply(extract_max_minutes)
df['autonomia'] = df['autonomia'].fillna(df['alcance'].apply(extract_max_minutes))


In [98]:
def extract_largest_side(dimension_string):
    """
    Extracts the largest side in centimeters from a dimension string.
    Handles formats like '30l. x 30an. x 7,8al. centímetros' and '50 x 50 x 28 cm'.
    """
    # Regex to capture numeric dimensions with possible Spanish abbreviations (l., an., al.)
    pattern = r"([\d.,]+)l?\.*\s*[xX]\s*([\d.,]+)a?n?\.*\s*[xX]\s*([\d.,]+)a?l?\.*.*?(cm|centímetros|milímetros)?"
    match = re.search(pattern, str(dimension_string), re.IGNORECASE)
    
    if match:
        # Extract the three dimensions
        dimensions = match.groups()[:3]
        # Convert dimensions to float
        dimensions = [float(dim.replace(',', '.')) for dim in dimensions]
        
        # Check the unit and convert to centimeters if needed
        if "milímetros" in dimension_string:
            dimensions = [dim / 10 for dim in dimensions]  # Convert mm to cm
        
        # Return the largest dimension
        return max(dimensions)
    else:
        return None  # Return

df["Altura (cm)"]=df['dimensiones del producto'].apply(extract_largest_side)

In [99]:
mapping = {
    r"(?i)alfombra|carpet": "Carpets",
    r"(?i)suelo duro|suelos duros|hardwood|hard floors": "Hard Floors",
    r"(?i)tapicería|upholstery": "Upholstery",
    r"(?i)cama|bed": "Beds",
    r"(?i)mueble|sofá|furniture": "Other Surfaces"
}

# Función para realizar el mapeo
def map_surface_type(description, mapping):
    categories = set()
    for pattern, category in mapping.items():
        if re.search(pattern, description):
            categories.add(category)
    return ", ".join(categories) if categories else "Other Surfaces"

# Function to combine both columns
    
df["tipos de superficie"]=df["tipos de superficie"].apply(lambda x: map_surface_type(str(x), mapping))
df["tipos de superficie"] = df.apply(
        lambda row: ", ".join(sorted(set(str(row["tipos de superficie"]).split(", ") + str(row["surface_types"]).split(", ")))),
        axis=1
    )
    
df["tipos de superficie"]=df["tipos de superficie"].apply(lambda x: str(x).replace("None, ","").replace(", None",""))



In [100]:
def classify_filter(description):
    """
    Classify the filter description into standardized categories.
    
    Parameters:
    description (str): The filter description.
    
    Returns:
    str: The classified category.
    """
    if not isinstance(description, str):
        return 'Other'
    
    # Normalize the description
    desc = remove_accents(description).lower()
    
    # Initialize category
    category = []
    
    # Check for HEPA
    if 'hepa' in desc:
        return "HEPA"
        #category.append('HEPA')
    
    ## Check for EPA
    #if 'epa' in desc.replace("hepa",""):
    #    category.append('EPA')
    #
    ## Check for washable
    #if 'lavable' in desc or 'washable' in desc:
    #    category.append('Washable')
    #
    ## Check for cyclonic
    #if 'ciclonic' in desc or 'cyclonic' in desc or 'cyclone' in desc:
    #    category.append('Cyclonic')
    #
    ## Check for multilayer
    #if 'multicapa' in desc or 'multilayer' in desc or 'multicapa' in desc or 'multicapa' in desc:
    #    category.append('Multilayer')
    #
    ## Check for reusable
    #if 'reutilizable' in desc or 'reusable' in desc:
    #    category.append('Reusable')
    #
    ## Combine categories or assign 'Other'
    #if category:
    #    return ' + '.join(category)
    else:
        return 'Other'



df['filtro']=df['filtro'].apply(classify_filter)
df.loc[df['filtro']=="Other",'filtro']=df.loc[df['filtro']=="Other","componentes"].apply(lambda x: "HEPA" if "hepa" in str(x).lower() else "Other")
df.loc[df['filtro']=="Other",'filtro']=df.loc[df['filtro']=="Other",'caracteristicas adicionales'].apply(lambda x: "HEPA" if "hepa" in str(x).lower() else "Other")





In [101]:
df['motor'].value_counts()

motor
Motor sin escobillas                                           4
450W                                                           4
Motor sin escobillas de 550 W                                  3
550 W                                                          3
550W                                                           3
160W                                                           3
120 W                                                          2
580W                                                           2
LongLife                                                       2
Sin escobillas                                                 2
Potente motor                                                  2
100 W                                                          2
Motor sin escobillas de 550W                                   2
alta velocidad                                                 1
potente motor sin escobillas de 26 KPA/250 W                   1
93 W               

In [102]:
fixed_columns=['Title', 'Stars', 'Price(€)',"peso","garantia","nivel de ruido (dB)",
"alimentacion","N_cepillos","type_cepillo", "features_cepillo","volume_litres",
"tiempo de carga","tipos de superficie","funcion seca suelo",'suction_pressure_kpa',
"autonomia","Altura (cm)","filtro",
"componentes","accesorios",'caracteristicas adicionales','All_info']

In [103]:
df=df.rename(columns={"Price":"Price(€)","nivel de ruido":"nivel de ruido (dB)"})

In [104]:
# "se incluyen las pilas" esperaba que fuera si/no, pero son textos :(
# material es 99% plástico, no aporta información
irrelevantes=["color","apariencia","material","marca",'modos de funcionamiento']
df=df.drop(columns=["bolsa de suciedad","tipos de filtros","ahorro de energia","nombre del modelo",'accesorios incluidos',
                    'tipos de accesorios',"tiene filtro",'ahorro energia',"tipos de suelo","fabricante",
                    "conectividad","se incluyen las pilas","longitud cable","clase de reemision de polvo","consumo","bateria","necesita baterias","cepillo",
                    "apagado automatico","aplicaciones","maxima presion de succion","cabezal","alcance",'dimensiones del producto',
                    "amperaje","surface_types",'tiene ahorro energia',"iluminacion",'detalles fisicos adicionales',
                    'motor',"alimentacion"]+irrelevantes+priority_order,errors="ignore")

In [105]:
df.to_csv('Databases/Prepared_df.csv',index=False)

In [106]:
pd.set_option('display.max_rows', None)